MSB-1 - Module 0: Basic Mechanics

In [8]:
import sympy as sp
import pint
import sympy as sp
from sympy.physics.units import *
import pint

from IPython.display import display, Math, HTML, Markdown
import inspect
from __future__ import annotations
import numpy as np
from constants import greek_letters
import re
import pandas as pd
import matplotlib

u = pint.UnitRegistry()
u.formatter.default_format= '~P'
Q_ = u.Quantity

# Silence NEP 18 warning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    Q_([])

In [9]:
#This block attempts to render equations, variable assignments, units and comments in LaTeX format 

DEBUG_MODE =  False #Set this to True to enable debug statements

def debug_print(*args, **kwargs):
    if DEBUG_MODE:
        print(*args, **kwargs)
        
def escape_latex(text):
    #Escape LaTeX special characters in the text, except underscores
    replacements = {
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '_': r'_',  # Keep underscore as is
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\textasciicircum{}',
        #'\\': r'\textbackslash{}',
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

def replace_greek_letters(text):
    # Use regex to replace whole words that match Greek letters
    pattern = r'(' + '|'.join(re.escape(key) for key in greek_letters.keys()) + r')'
    debug_print(f"Pattern: {pattern}")
    greek_sub = re.sub(pattern, lambda m: greek_letters[m.group(1)], text)
    debug_print(f"Replaced Greek letters: {greek_sub}")
    return greek_sub

def format_var_name(name):
    # Capture first underscore in variable name so that subscript can be shown correctly
    if '_' in name:
        base, subscript = name.split('_', 1)
        debug_print(f"Base is : {base}, subscript is : {subscript}")
        base = replace_greek_letters(base)
        subscript = replace_greek_letters(subscript) 
        subscript = escape_latex(subscript)
        debug_print(f"After escaping, subscript is : {subscript}")
        formatted_name = f"{base}_{{\\text{{{subscript}}}}}"
    else:
        name = replace_greek_letters(name)
        formatted_name = escape_latex(name)
    return formatted_name

def capture_var_name(func):
    #Capture the variable name of the first argument passed to the function
    def wrapper(*args, **kwargs):
        frame = inspect.currentframe().f_back
        var_name = [name for name, val in frame.f_locals.items() if val is args[0]][0]
        debug_print(f"Captured variable name: {var_name}")
        return func(var_name, *args, **kwargs)
    return wrapper

@capture_var_name
def displaymath(var_name, expr, comment='', comment_size="small", equation_size="small", line_height="1.2", comment_width="50%"):
    #Generate LaTeX code for the expression and display it
    # Debug debug_print statements
    debug_print(f"Input expression: {expr}")
    debug_print(f"Type of expression: {type(expr)}")
    debug_print(f"Variable name: {var_name}")
    # Check if expr is a SymPy expression
    if isinstance(expr, sp.Basic):
        debug_print("Expression is a SymPy Basic type.")
        if isinstance(expr, sp.Matrix):
            debug_print("Expression is a SymPy Matrix.")
            # If it's a SymPy matrix, format it as an equation
            formatted_var_name = format_var_name(var_name)
            expr = replace_greek_letters(expr)
            equation_latex = f"{formatted_var_name} = {sp.latex(expr)}"
        else:
            debug_print("Expression is a SymPy expression but not a Matrix.")
            # If it's another SymPy expression, format it as an equation
            formatted_var_name = format_var_name(var_name)
            expr = replace_greek_letters(expr)
            debug_print(f"Formatted variable name: {formatted_var_name}")
            debug_print(f"Formatted expression: {expr}")
            equation_latex = f"{formatted_var_name} = {sp.latex(expr)}"
            debug_print(f"Equation LaTeX: {equation_latex}")
    elif isinstance(expr, sp.Matrix):
        debug_print("Expression is a SymPy Matrix with units.")
        # If it's a SymPy matrix with units, format each element
        formatted_var_name = format_var_name(var_name)
        matrix_latex = replace_greek_letters(sp.latex(expr.applyfunc(lambda x: x)))
        equation_latex = f"{formatted_var_name} = {matrix_latex}"
    elif isinstance(expr, u.Quantity):
        debug_print("Expression is a pint Quantity.")
        if isinstance(expr.magnitude, np.ndarray):
            debug_print("Magnitude is a NumPy array.")
            # If it's a NumPy array, format it as an equation
            formatted_var_name = format_var_name(var_name)
            equation_latex = f"{formatted_var_name} = {sp.latex(Matrix(expr.magnitude))} \\, {sp.latex(expr.units)}"
        else:
            debug_print("Magnitude is not a NumPy array.")
            # If it's a pint Quantity, format it as an equation
            formatted_var_name = format_var_name(var_name)
            equation_latex = f"{formatted_var_name} = {sp.latex(expr.magnitude)} \\, {sp.latex(expr.units)}"
    else:
        debug_print("Expression is a regular variable.")
        # expr is a regular variable
        formatted_var_name = format_var_name(var_name)
        if isinstance(expr, (int, float)):
            value_latex = sp.latex(expr)
        else:
            value_latex = str(expr)
        units = ''  # Add units here if necessary
        equation_latex = f"{formatted_var_name} = {value_latex} {units}"

    # Debug debug_print statement
    debug_print(f"Generated LaTeX: {equation_latex}")
    render_content(equation_latex, comment=comment, content_type='latex', equation_size=equation_size, comment_size=comment_size, line_height=line_height, comment_width=comment_width)

def render_content(content, comment='', content_type='latex', equation_size='small', comment_size='small', line_height='1.2', comment_width='50%'):
    """
    Renders LaTeX equations or HTML content with an optional comment.

    Parameters:
    - content (str): LaTeX code or HTML content to display.
    - comment (str): Optional comment to display alongside the content.
    - content_type (str): 'latex' or 'html' to specify the type of content.
    - equation_size (str): CSS font size for the content.
    - comment_size (str): CSS font size for the comment.
    - line_height (str): Line height for the text.
    - comment_width (str): Width of the comment area.
    """
    if content_type == 'latex':
        # Wrap LaTeX content with MathJax delimiters
                content_html = rf"\[ {content} \]"
    elif content_type == 'html':
        # Use the HTML content directly
        content_html = content
    else:
        raise ValueError("content_type must be 'latex' or 'html'")

    # Create the HTML code
    html_code = rf"""
    <script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML"></script>
    <script type="text/javascript">
         MathJax.Hub.Queue(["Typeset", MathJax.Hub]);
    </script>
        <div class="math">
            <div class="math-equation">
             {content_html}
            </div>
            <div class="math-comment">
             {comment}
            </div>
        </div>
    """

    #<div style="display: flex; flex-flow: row wrap; flex: 3; justify-content: space-between; align-items: center; width: 100%;">
    #</div>

    #<div style="font-size: {equation_size}; line-height: {line_height};">
    #<div style="font-size: {comment_size}; font-style: italic; line-height: {line_height}; text-align: right; max-width: {comment_width}; word-wrap: break-word; white-space: normal;">

    # If LaTeX content is included, ensure MathJax processes it
    ''' 
    if content_type == 'latex':
        html_code += """
        <script type="text/javascript" async
        src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
        </script>
        <script type="text/javascript">
            MathJax.Hub.Queue(["Typeset", MathJax.Hub]);
        </script>
        """
    '''
    # Display the HTML code
    display(HTML(html_code))

In [10]:
q_1 = Q_(0.3, 'm')
q_2 = Q_(0.4, 'm')
dq_1_dt = Q_(-1, 'm/s')
dq_2_dt = Q_(-0.4, 'm/s')
K_1 = Q_(25, 'N/m')
K_2 = Q_(10, 'N/m')
l_1 = Q_(0.1, 'm')
l_2 = Q_(0.2, 'm')
D_1 = Q_(2, 'N*s/m')
D_2 = Q_(5, 'N*s/m')
m_1 = Q_(5, 'kg')
m_2 = Q_(3, 'kg')
g = Q_(9.81, 'm/s^2')


In [13]:
t = sp.symbols('t')

F_1 = -K_1*(q_1-l_1) - D_1*dq_1_dt
displaymath(F_1)
F_2 = K_2*(q_2-l_2) + D_2*dq_2_dt
displaymath(F_2)

d2q_1_dt2 = (-F_1 - F_2 - m_1*g)/m_1.to('m/s^2')
displaymath(d2q_1_dt2)


DimensionalityError: Cannot convert from 'kilogram' ([mass]) to 'meter / second ** 2' ([length] / [time] ** 2)